In [6]:
from pathlib import Path
from meteostat import Daily, Point
from tqdm import tqdm
import datetime as dt
import pandas as Pd
import numpy as np
import os

# Automatically find project root (the one containing the 'data' folder)
ROOT = Path.cwd()
if not (ROOT / "data").exists():
    ROOT = ROOT.parent
os.chdir(ROOT)

DATA_DIR = Path("data")
drivers_Path = DATA_DIR/"drivers.csv"
Lap_Times_path = DATA_DIR/"lap_times.csv"
races_Path = DATA_DIR/"races.csv"
circuit_Path = DATA_DIR/"circuits.csv"
qual_Path = DATA_DIR/"qualifying.csv"

drivers = Pd.read_csv(drivers_Path)
lap_times = Pd.read_csv(Lap_Times_path)
races = Pd.read_csv(races_Path)
circuits = Pd.read_csv(circuit_Path)
qual = Pd.read_csv(qual_Path)

drv = drivers.copy()
drv["drivers_name"] = drv["forename"].str.strip() + " " + drv["surname"].str.strip()

best_laps = (
    lap_times
    .groupby(["raceId", "driverId"])["milliseconds"]
    .min()
    .reset_index()
    .rename(columns={"milliseconds":"bestLap_ms"})
)

best_laps["bestLaps_s"] = best_laps["bestLap_ms"]/1000.0
best_laps_named = best_laps.merge(drv[["driverId", "drivers_name"]], on="driverId", how="left")

races_small = races[["raceId", "year", "round", "circuitId", "name", "date"]].copy()
races_small.rename(columns={"name": "gp_name"}, inplace=True)

circuit_small = circuits[["circuitId", "name", "country", "lat", "lng", "alt"]].copy()
circuit_small.rename(columns={"name": "circuit_name"}, inplace=True)

bl = best_laps_named.copy()
bl_race = bl.merge(races_small, on="raceId", how="left")
bl_full = bl_race.merge(circuit_small, on="circuitId", how="left")

def time_str_to_seconds(x):
    """
    Convert time strings like '1:12.345' to seconds (float).
    Handles None/NaN/empty safely.
    """
    if Pd.isna(x) or x == "":
        return np.nan
    s = str(x)
    if ":" in s:
        m, rest = s.split(":")
        return float(m) * 60.0 + float(rest)
    # sometimes it's already seconds-like
    try:
        return float(s)
    except Exception:
        return np.nan

# apply to each q-column
for col in ["q1", "q2", "q3"]:
    if col in qual.columns:
        qual[col + "_s"] = qual[col].apply(time_str_to_seconds)

# best of the available sessions (some drivers only set Q1 or Q2)
q_time_cols = [c for c in ["q1_s","q2_s","q3_s"] if c in qual.columns]
qual["qual_best_s"] = qual[q_time_cols].min(axis=1, skipna=True)

# keep only the fields we need
qual_small = qual[["raceId","driverId","position","qual_best_s"]].copy()
qual_small.rename(columns={"position":"qual_pos"}, inplace=True)

# bl_full currently has: [raceId, driverId, bestLap_s, year, gp_name, circuit_name, country, lat, lng, ...]
bl_q = bl_full.merge(qual_small, on=["raceId","driverId"], how="left")

def get_weather(lat, lng, date_str):
    """
    Fetch weather for given location (lat, lng) and race date.
    Returns a dict with average temperature, rain, wind speed, and pressure.
    """
    try:
        date = Pd.to_datetime(date_str)
        start = date - dt.timedelta(days=1)  # buffer (previous day)
        end = date + dt.timedelta(days=1)
        location = Point(lat, lng)
        data = Daily(location, start, end)
        df = data.fetch()
        if df.empty:
            return {"temp_c": None, "rain_mm": None, "wind_mps": None, "pressure_hpa": None}
        # Take mean of these 3 days for smoother approximation
        return {
            "temp_c": round(df["tavg"].mean(), 2) if "tavg" in df else None,
            "rain_mm": round(df["prcp"].sum(), 2) if "prcp" in df else None,
            "wind_mps": round(df["wspd"].mean(), 2) if "wspd" in df else None,
            "pressure_hpa": round(df["pres"].mean(), 2) if "pres" in df else None,
        }
    except Exception as e:
        print("⚠️ Error:", e)
        return {"temp_c": None, "rain_mm": None, "wind_mps": None, "pressure_hpa": None}

weather_cache = {}
weather_records = []

for _, row in tqdm(bl_q.iterrows(), total=len(bl_q)):
    race_id = row["raceId"]
    if race_id not in weather_cache:
        w = get_weather(row["lat"], row["lng"], row["date"])
        weather_cache[race_id] = w
    weather_records.append(weather_cache[race_id])

weather_df = Pd.DataFrame(weather_records)
bl_weather = Pd.concat([bl_q.reset_index(drop=True), weather_df], axis=1)

bl_weather.to_csv("outputs/f1_features_weather.csv", index=False)


  0%|          | 20/11041 [00:00<02:14, 81.67it/s]

⚠️ Error: type NAType doesn't define __round__ method
⚠️ Error: type NAType doesn't define __round__ method


  1%|          | 59/11041 [00:00<01:21, 134.75it/s]

⚠️ Error: type NAType doesn't define __round__ method
⚠️ Error: type NAType doesn't define __round__ method


  1%|          | 135/11041 [00:00<01:05, 166.64it/s]

...

 80%|████████  | 8880/11041 [00:50<00:12, 174.30it/s]

⚠️ Error: type NAType doesn't define __round__ method


 82%|████████▏ | 9000/11041 [00:51<00:11, 182.14it/s]

⚠️ Error: type NAType doesn't define __round__ method


100%|██████████| 11041/11041 [01:03<00:00, 174.27it/s]
